In [1]:
import os
import re
import json
import pandas as pd
from copy import deepcopy
from collections import defaultdict
from typing import Any, Dict, List

In [2]:
graph = {}
with open(f'../../data/mlb/computes/team_event_graph.json', 'r', encoding='UTF8') as pbp_input:
    graph = json.load(pbp_input)

In [3]:
def compute_likelihoods(graph):
    graph_copy = deepcopy(graph)
    for team in graph_copy.keys():
        for area in graph_copy[team].keys():
            for out in graph_copy[team][area].keys():
                for state in graph_copy[team][area][out].keys():
                    runs = graph_copy[team][area][out][state]['runs']
                    total = sum(graph_copy[team][area][out][state]['types'].values())
                    graph_copy[team][area][out][state] = 0.0 if runs == 0 else round(float(runs) / total, 3)
    
    return graph_copy

likelihood_graph = compute_likelihoods(graph)
likelihood_graph

{'ari': {'home': {'0': {'000': 0.074,
    '100': 0.0,
    '010': 0.0,
    '110': 2.0,
    '001': 0.0,
    '101': 0.0,
    '011': 0.0,
    '111': 0.0},
   '1': {'000': 0.042,
    '100': 0.5,
    '010': 0.0,
    '110': 0.0,
    '001': 1.0,
    '101': 0.0,
    '011': 0.0,
    '111': 1.667},
   '2': {'000': 0.045,
    '100': 0.0,
    '010': 0.0,
    '110': 0.0,
    '001': 0.0,
    '101': 0.0,
    '011': 0.0,
    '111': 0.0}},
  'away': {'0': {'000': 0.0,
    '100': 0.0,
    '010': 0.0,
    '110': 0.0,
    '001': 0.0,
    '101': 0.0,
    '011': 0.0,
    '111': 0.0},
   '1': {'000': 0.0,
    '100': 0.0,
    '010': 0.0,
    '110': 0.0,
    '001': 0.0,
    '101': 0.0,
    '011': 0.0,
    '111': 0.0},
   '2': {'000': 0.0,
    '100': 0.0,
    '010': 0.0,
    '110': 0.0,
    '001': 0.0,
    '101': 0.0,
    '011': 0.0,
    '111': 0.0}}},
 'tb': {'home': {'0': {'000': 0.0,
    '100': 0.0,
    '010': 0.25,
    '110': 0.0,
    '001': 0.0,
    '101': 3.0,
    '011': 0.0,
    '111': 0.0},
   '1': {'000

In [4]:
def flatten_graph(graph):
    records = []
    for team in graph.keys():  
        for where in graph[team].keys():
            for outs in graph[team][where].keys():
                row = {
                    'team': team,
                    'where': where,
                    'team': team,
                    'outs': outs,
                }

                row.update(
                    graph[team][where][outs]
                )

                records.append(row)

    return records

def flatten_full_graph(graph):
    records = []
    for team in graph.keys():  
        for where in graph[team].keys():
            for outs in graph[team][where].keys():
                for bases in graph[team][where][outs].keys():
                    row = {
                        'team': team,
                        'where': where,
                        'team': team,
                        'outs': outs,
                        'bases': bases
                    }

                    row.update(
                        graph[team][where][outs][bases]['types']
                    )

                    records.append(row)

    return records

In [5]:
pd.DataFrame(flatten_full_graph(graph))

,team,where,outs,bases,struck out,popped out,lined out,tripled,grounded out,fouled out,...,hit by pitch,throwing error,lined into double play,fielding error,catcher's interference,flied into double play,popped into double play,intentionally walked,error,singles
0,ari,home,0,000,3.0,1.0,1.0,1.0,5.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ari,home,0,100,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ari,home,0,010,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ari,home,0,110,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ari,home,0,001,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,sea,away,2,110,1.0,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,sea,away,2,001,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
765,sea,away,2,101,2.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,sea,away,2,011,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pd.DataFrame(flatten_graph(likelihood_graph))

,team,where,outs,000,100,010,110,001,101,011,111
0,ari,home,0,0.074,0.000,0.000,2.00,0.000,0.0,0.0,0.000
1,ari,home,1,0.042,0.500,0.000,0.00,1.000,0.0,0.0,1.667
2,ari,home,2,0.045,0.000,0.000,0.00,0.000,0.0,0.0,0.000
3,ari,away,0,0.000,0.000,0.000,0.00,0.000,0.0,0.0,0.000
4,ari,away,1,0.000,0.000,0.000,0.00,0.000,0.0,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...,...
91,sea,home,1,0.048,0.000,0.667,0.00,0.000,1.0,0.0,0.000
92,sea,home,2,0.000,0.000,0.400,0.00,0.000,0.0,0.0,0.000
93,sea,away,0,0.000,0.000,0.000,0.25,0.000,0.0,0.0,0.000
94,sea,away,1,0.037,0.000,0.000,0.00,0.000,0.0,0.0,0.000
